In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, np_utils
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model, model_from_json
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
import pandas as pd
import librosa
import glob 
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
mylist= os.listdir(r'C:\Users\DC\Desktop\virtual\speechemotion\')

In [3]:
feeling_list=[]
for item in mylist:
    if item[6:-16]=='03' and int(item[18:-4])%2==0:
        feeling_list.append('female_happy')
    elif item[6:-16]=='03' and int(item[18:-4])%2==1:
        feeling_list.append('male_happy')
    elif item[6:-16]=='04' and int(item[18:-4])%2==0:
        feeling_list.append('female_sad')
    elif item[6:-16]=='04' and int(item[18:-4])%2==1:
        feeling_list.append('male_sad')
    elif item[6:-16]=='05' and int(item[18:-4])%2==0:
        feeling_list.append('female_angry')
    elif item[6:-16]=='05' and int(item[18:-4])%2==1:
        feeling_list.append('male_angry')
    elif item[:1]=='a':
        feeling_list.append('male_angry')
    elif item[:1]=='h':
        feeling_list.append('male_happy')
    elif item[:2]=='sa':
        feeling_list.append('male_sad')

In [4]:
labels = pd.DataFrame(feeling_list)
print(labels.head())

              0
0    male_happy
1  female_happy
2    male_happy
3  female_happy
4    male_happy


In [ ]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]=='03' or mylist[index][6:-16]=='04' or mylist[index][6:-16]=='05' or mylist[index][:2]=='a' or mylist[index][:1]=='h' or mylist[index][:1]=='sa':
        X, sample_rate = librosa.load('E:/SpeechEmotion/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1

In [12]:
df3 = pd.DataFrame(df['feature'].values.tolist())
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf = shuffle(newdf)
rnewdf=rnewdf.fillna(0)

In [19]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]
print(testlabel)

                0
479  female_angry
71   female_happy
52     male_happy
517  female_angry
693    male_happy
..            ...
491  female_angry
613    male_angry
627    male_angry
218      male_sad
348      male_sad

[153 rows x 1 columns]


In [20]:
X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)
lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [21]:
X_train.shape

(603, 216)

In [22]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [26]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 216, 256)          1536      
_________________________________________________________________
activation_6 (Activation)    (None, 216, 256)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 216, 128)          163968    
_________________________________________________________________
activation_7 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 27, 128)          

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [30]:
checkpoint = ModelCheckpoint(r"E:\demo1\speeash.h5", monitor='val_accuracy',save_weights_only=True, mode='max', verbose=1,
                             save_best_only=True)
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=700, validation_data=(x_testcnn, y_test),callbacks=[checkpoint])

fer_json = model.to_json()
with open("E:\demo1\speeash.json", "w") as json_file:
    json_file.write(fer_json)

Train on 603 samples, validate on 153 samples
Epoch 1/700
603/603 [==============================] - ETA: 6s - loss: 1.5923 - accuracy: 0.50 - ETA: 6s - loss: 1.5473 - accuracy: 0.43 - ETA: 6s - loss: 1.4958 - accuracy: 0.52 - ETA: 6s - loss: 1.5137 - accuracy: 0.53 - ETA: 6s - loss: 1.5059 - accuracy: 0.52 - ETA: 6s - loss: 1.5291 - accuracy: 0.48 - ETA: 6s - loss: 1.5416 - accuracy: 0.49 - ETA: 5s - loss: 1.5530 - accuracy: 0.46 - ETA: 5s - loss: 1.5592 - accuracy: 0.44 - ETA: 5s - loss: 1.5539 - accuracy: 0.45 - ETA: 5s - loss: 1.5647 - accuracy: 0.45 - ETA: 5s - loss: 1.5677 - accuracy: 0.44 - ETA: 4s - loss: 1.5783 - accuracy: 0.43 - ETA: 4s - loss: 1.5772 - accuracy: 0.44 - ETA: 4s - loss: 1.5782 - accuracy: 0.43 - ETA: 4s - loss: 1.5796 - accuracy: 0.43 - ETA: 4s - loss: 1.5769 - accuracy: 0.42 - ETA: 3s - loss: 1.5825 - accuracy: 0.40 - ETA: 3s - loss: 1.5820 - accuracy: 0.41 - ETA: 3s - loss: 1.5833 - accuracy: 0.40 - ETA: 3s - loss: 1.5834 - accuracy: 0.39 - ETA: 3s - loss: 1

603/603 [==============================] - ETA: 6s - loss: 1.3727 - accuracy: 0.50 - ETA: 6s - loss: 1.3568 - accuracy: 0.50 - ETA: 6s - loss: 1.3480 - accuracy: 0.50 - ETA: 6s - loss: 1.3749 - accuracy: 0.46 - ETA: 6s - loss: 1.3389 - accuracy: 0.48 - ETA: 6s - loss: 1.3555 - accuracy: 0.47 - ETA: 6s - loss: 1.3574 - accuracy: 0.49 - ETA: 5s - loss: 1.3632 - accuracy: 0.49 - ETA: 5s - loss: 1.3892 - accuracy: 0.46 - ETA: 5s - loss: 1.4063 - accuracy: 0.45 - ETA: 5s - loss: 1.4114 - accuracy: 0.46 - ETA: 4s - loss: 1.4115 - accuracy: 0.46 - ETA: 4s - loss: 1.4016 - accuracy: 0.48 - ETA: 4s - loss: 1.4071 - accuracy: 0.49 - ETA: 4s - loss: 1.4105 - accuracy: 0.48 - ETA: 4s - loss: 1.4182 - accuracy: 0.47 - ETA: 4s - loss: 1.4227 - accuracy: 0.47 - ETA: 3s - loss: 1.4177 - accuracy: 0.47 - ETA: 3s - loss: 1.4212 - accuracy: 0.47 - ETA: 3s - loss: 1.4168 - accuracy: 0.48 - ETA: 3s - loss: 1.4145 - accuracy: 0.48 - ETA: 3s - loss: 1.4138 - accuracy: 0.49 - ETA: 2s - loss: 1.4115 - accuracy

603/603 [==============================] - ETA: 6s - loss: 1.3730 - accuracy: 0.31 - ETA: 6s - loss: 1.3129 - accuracy: 0.50 - ETA: 7s - loss: 1.3335 - accuracy: 0.52 - ETA: 6s - loss: 1.3628 - accuracy: 0.53 - ETA: 6s - loss: 1.3563 - accuracy: 0.51 - ETA: 6s - loss: 1.3170 - accuracy: 0.54 - ETA: 5s - loss: 1.3067 - accuracy: 0.54 - ETA: 5s - loss: 1.2957 - accuracy: 0.54 - ETA: 5s - loss: 1.2924 - accuracy: 0.54 - ETA: 5s - loss: 1.2867 - accuracy: 0.53 - ETA: 5s - loss: 1.2862 - accuracy: 0.55 - ETA: 5s - loss: 1.2844 - accuracy: 0.54 - ETA: 4s - loss: 1.2751 - accuracy: 0.53 - ETA: 4s - loss: 1.2543 - accuracy: 0.54 - ETA: 4s - loss: 1.2751 - accuracy: 0.53 - ETA: 4s - loss: 1.2706 - accuracy: 0.53 - ETA: 4s - loss: 1.2807 - accuracy: 0.52 - ETA: 3s - loss: 1.2849 - accuracy: 0.52 - ETA: 3s - loss: 1.2782 - accuracy: 0.52 - ETA: 3s - loss: 1.2739 - accuracy: 0.52 - ETA: 3s - loss: 1.2709 - accuracy: 0.52 - ETA: 3s - loss: 1.2748 - accuracy: 0.52 - ETA: 2s - loss: 1.2754 - accuracy

603/603 [==============================] - ETA: 6s - loss: 1.0388 - accuracy: 0.62 - ETA: 6s - loss: 1.1258 - accuracy: 0.59 - ETA: 6s - loss: 1.0433 - accuracy: 0.62 - ETA: 6s - loss: 1.0776 - accuracy: 0.62 - ETA: 6s - loss: 1.0846 - accuracy: 0.63 - ETA: 6s - loss: 1.1023 - accuracy: 0.60 - ETA: 6s - loss: 1.0913 - accuracy: 0.62 - ETA: 5s - loss: 1.1026 - accuracy: 0.61 - ETA: 5s - loss: 1.1228 - accuracy: 0.59 - ETA: 5s - loss: 1.1342 - accuracy: 0.56 - ETA: 5s - loss: 1.1388 - accuracy: 0.56 - ETA: 5s - loss: 1.1530 - accuracy: 0.55 - ETA: 4s - loss: 1.1634 - accuracy: 0.54 - ETA: 4s - loss: 1.1687 - accuracy: 0.53 - ETA: 4s - loss: 1.1677 - accuracy: 0.52 - ETA: 4s - loss: 1.1646 - accuracy: 0.51 - ETA: 4s - loss: 1.1684 - accuracy: 0.52 - ETA: 3s - loss: 1.1629 - accuracy: 0.52 - ETA: 3s - loss: 1.1512 - accuracy: 0.53 - ETA: 3s - loss: 1.1504 - accuracy: 0.53 - ETA: 3s - loss: 1.1445 - accuracy: 0.54 - ETA: 3s - loss: 1.1388 - accuracy: 0.55 - ETA: 2s - loss: 1.1380 - accuracy

603/603 [==============================] - ETA: 6s - loss: 0.9538 - accuracy: 0.68 - ETA: 6s - loss: 0.9434 - accuracy: 0.65 - ETA: 6s - loss: 1.0480 - accuracy: 0.56 - ETA: 6s - loss: 1.0136 - accuracy: 0.59 - ETA: 6s - loss: 0.9525 - accuracy: 0.63 - ETA: 6s - loss: 0.9893 - accuracy: 0.63 - ETA: 5s - loss: 1.0052 - accuracy: 0.62 - ETA: 5s - loss: 1.0201 - accuracy: 0.61 - ETA: 5s - loss: 1.0311 - accuracy: 0.61 - ETA: 5s - loss: 1.0191 - accuracy: 0.61 - ETA: 5s - loss: 1.0623 - accuracy: 0.59 - ETA: 4s - loss: 1.0728 - accuracy: 0.58 - ETA: 4s - loss: 1.0748 - accuracy: 0.58 - ETA: 4s - loss: 1.0679 - accuracy: 0.58 - ETA: 4s - loss: 1.0546 - accuracy: 0.57 - ETA: 4s - loss: 1.0418 - accuracy: 0.58 - ETA: 4s - loss: 1.0429 - accuracy: 0.58 - ETA: 3s - loss: 1.0392 - accuracy: 0.57 - ETA: 3s - loss: 1.0437 - accuracy: 0.57 - ETA: 3s - loss: 1.0468 - accuracy: 0.56 - ETA: 3s - loss: 1.0462 - accuracy: 0.56 - ETA: 3s - loss: 1.0442 - accuracy: 0.56 - ETA: 2s - loss: 1.0521 - accuracy

...

603/603 [==============================] - ETA: 6s - loss: 0.2470 - accuracy: 0.93 - ETA: 6s - loss: 0.2728 - accuracy: 0.96 - ETA: 6s - loss: 0.3258 - accuracy: 0.93 - ETA: 6s - loss: 0.3137 - accuracy: 0.93 - ETA: 6s - loss: 0.3562 - accuracy: 0.91 - ETA: 6s - loss: 0.3526 - accuracy: 0.90 - ETA: 6s - loss: 0.3717 - accuracy: 0.89 - ETA: 6s - loss: 0.3920 - accuracy: 0.85 - ETA: 5s - loss: 0.3828 - accuracy: 0.87 - ETA: 5s - loss: 0.3772 - accuracy: 0.88 - ETA: 5s - loss: 0.3667 - accuracy: 0.87 - ETA: 5s - loss: 0.3774 - accuracy: 0.85 - ETA: 5s - loss: 0.3653 - accuracy: 0.86 - ETA: 4s - loss: 0.3607 - accuracy: 0.86 - ETA: 4s - loss: 0.3612 - accuracy: 0.85 - ETA: 4s - loss: 0.3709 - accuracy: 0.85 - ETA: 4s - loss: 0.3642 - accuracy: 0.86 - ETA: 4s - loss: 0.3688 - accuracy: 0.86 - ETA: 3s - loss: 0.3817 - accuracy: 0.85 - ETA: 3s - loss: 0.3776 - accuracy: 0.85 - ETA: 3s - loss: 0.3685 - accuracy: 0.85 - ETA: 3s - loss: 0.3618 - accuracy: 0.86 - ETA: 3s - loss: 0.3561 - accuracy

603/603 [==============================] - ETA: 7s - loss: 0.3873 - accuracy: 0.68 - ETA: 7s - loss: 0.4752 - accuracy: 0.68 - ETA: 7s - loss: 0.5167 - accuracy: 0.68 - ETA: 6s - loss: 0.4542 - accuracy: 0.73 - ETA: 6s - loss: 0.4080 - accuracy: 0.77 - ETA: 6s - loss: 0.4064 - accuracy: 0.79 - ETA: 6s - loss: 0.4184 - accuracy: 0.77 - ETA: 6s - loss: 0.4033 - accuracy: 0.79 - ETA: 6s - loss: 0.3877 - accuracy: 0.81 - ETA: 5s - loss: 0.3840 - accuracy: 0.83 - ETA: 5s - loss: 0.3697 - accuracy: 0.83 - ETA: 5s - loss: 0.3550 - accuracy: 0.84 - ETA: 5s - loss: 0.3597 - accuracy: 0.83 - ETA: 4s - loss: 0.3641 - accuracy: 0.83 - ETA: 4s - loss: 0.3642 - accuracy: 0.83 - ETA: 4s - loss: 0.3602 - accuracy: 0.83 - ETA: 4s - loss: 0.3575 - accuracy: 0.83 - ETA: 4s - loss: 0.3519 - accuracy: 0.83 - ETA: 3s - loss: 0.3534 - accuracy: 0.83 - ETA: 3s - loss: 0.3455 - accuracy: 0.84 - ETA: 3s - loss: 0.3420 - accuracy: 0.84 - ETA: 3s - loss: 0.3407 - accuracy: 0.84 - ETA: 3s - loss: 0.3449 - accuracy

603/603 [==============================] - ETA: 7s - loss: 0.3418 - accuracy: 0.87 - ETA: 7s - loss: 0.2784 - accuracy: 0.90 - ETA: 7s - loss: 0.3081 - accuracy: 0.85 - ETA: 6s - loss: 0.3109 - accuracy: 0.87 - ETA: 6s - loss: 0.3007 - accuracy: 0.87 - ETA: 6s - loss: 0.3009 - accuracy: 0.86 - ETA: 6s - loss: 0.3046 - accuracy: 0.87 - ETA: 6s - loss: 0.3153 - accuracy: 0.86 - ETA: 6s - loss: 0.3147 - accuracy: 0.86 - ETA: 5s - loss: 0.3102 - accuracy: 0.87 - ETA: 5s - loss: 0.3121 - accuracy: 0.87 - ETA: 5s - loss: 0.3169 - accuracy: 0.86 - ETA: 5s - loss: 0.3202 - accuracy: 0.86 - ETA: 5s - loss: 0.3283 - accuracy: 0.86 - ETA: 4s - loss: 0.3210 - accuracy: 0.87 - ETA: 4s - loss: 0.3219 - accuracy: 0.87 - ETA: 4s - loss: 0.3275 - accuracy: 0.86 - ETA: 4s - loss: 0.3323 - accuracy: 0.85 - ETA: 3s - loss: 0.3298 - accuracy: 0.85 - ETA: 3s - loss: 0.3251 - accuracy: 0.86 - ETA: 3s - loss: 0.3280 - accuracy: 0.86 - ETA: 3s - loss: 0.3286 - accuracy: 0.86 - ETA: 3s - loss: 0.3282 - accuracy

603/603 [==============================] - ETA: 5s - loss: 0.3116 - accuracy: 0.87 - ETA: 5s - loss: 0.2767 - accuracy: 0.90 - ETA: 5s - loss: 0.2818 - accuracy: 0.89 - ETA: 5s - loss: 0.2576 - accuracy: 0.92 - ETA: 5s - loss: 0.2466 - accuracy: 0.93 - ETA: 5s - loss: 0.2403 - accuracy: 0.94 - ETA: 4s - loss: 0.2661 - accuracy: 0.91 - ETA: 4s - loss: 0.2938 - accuracy: 0.89 - ETA: 4s - loss: 0.3014 - accuracy: 0.87 - ETA: 4s - loss: 0.3021 - accuracy: 0.86 - ETA: 4s - loss: 0.3249 - accuracy: 0.84 - ETA: 4s - loss: 0.3193 - accuracy: 0.85 - ETA: 3s - loss: 0.3208 - accuracy: 0.85 - ETA: 3s - loss: 0.3224 - accuracy: 0.85 - ETA: 3s - loss: 0.3153 - accuracy: 0.85 - ETA: 3s - loss: 0.3137 - accuracy: 0.85 - ETA: 3s - loss: 0.3196 - accuracy: 0.85 - ETA: 3s - loss: 0.3286 - accuracy: 0.84 - ETA: 2s - loss: 0.3281 - accuracy: 0.84 - ETA: 2s - loss: 0.3229 - accuracy: 0.85 - ETA: 2s - loss: 0.3200 - accuracy: 0.86 - ETA: 2s - loss: 0.3141 - accuracy: 0.86 - ETA: 2s - loss: 0.3048 - accuracy

KeyboardInterrupt: 

In [31]:
fer_json = model.to_json()
with open("E:\demo1\speeash.json", "w") as json_file:
    json_file.write(fer_json)